In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
cd /content/drive/MyDrive/Reimplementation/MViT/mvit

/content/drive/MyDrive/Reimplementation/MViT/mvit


In [12]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb  9 22:44:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
pip install fvcore fairscale simplejson psutil iopath

In [14]:
import torch
import mvit.models.attention
from mvit.models.mvit_model import PatchEmbed as PatchEmbed_3D
from mvit.models.mvit_model_2d import PatchEmbed as PatchEmbed_2D
import mvit.models.common
import demo_setup.pseudo_cfg_for_demo
import demo_setup.pseudo_defaults_for_demo

In [15]:
patch_embedder_3D = PatchEmbed_3D(dim_out = 96, padding=(0, 3, 3))
patch_embedder_2D = PatchEmbed_2D(dim_out = 96)

model_3d_dict = patch_embedder_3D.state_dict()
model_2d_dict = patch_embedder_2D.state_dict()

In [16]:
weight_pretrained = torch.load("/content/drive/MyDrive/Reimplementation/MViT/mvit/check_points/MViTv2_T_in1k.pyth")['model_state']

match_2d = {}
match_3d = {}

for k, v in weight_pretrained.items():
  if (k == "patch_embed.proj.weight"):
    match_2d["proj.weight"] = v
    match_3d["proj.weight"] = v.unsqueeze(2)

  if (k == "patch_embed.proj.bias"):
    match_2d["proj.bias"] = v
    match_3d["proj.bias"] = v


model_2d_dict.update(match_2d)
patch_embedder_2D.load_state_dict(model_2d_dict)

model_3d_dict.update(match_3d)
patch_embedder_3D.load_state_dict(model_3d_dict)

<ipython-input-16-b4c48271a164>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weight_pretrained = torch.load("/content/drive/MyDrive/Reimplementation/MViT/mvit/check_poin

<All keys matched successfully>

In [17]:
v1 = []
v2 = []
parameter_name = ["patch_embed.proj.weight", "patch_embed.proj.bias"]

for k,v in patch_embedder_3D.named_parameters():
  v1.append(v.flatten())

for k,v in patch_embedder_2D.named_parameters():
  v2.append(v.flatten())


for i in range(2):
  print("Comparing 2D with 3D, flatten parameters in", parameter_name[i], "are the same? ", torch.allclose(v1[i], v2[i], atol=1e-6))

# All parameters are the same!

Comparing 2D with 3D, flatten parameters in patch_embed.proj.weight are the same?  True
Comparing 2D with 3D, flatten parameters in patch_embed.proj.bias are the same?  True


In [18]:
# Generate same input for both (Conv3D expects 5D, Conv2D expects 4D)
x_3d = torch.randn(1, 3, 1, 224, 224)  # Depth=1
x_2d = x_3d.squeeze(2)

out_3d = patch_embedder_3D(x_3d)[0].squeeze(2)
out_2d = patch_embedder_2D(x_2d)[0]

print("2D patch embedding and 3D patch embedding have the same output?", torch.allclose(out_3d, out_2d, atol=1e-6))

# Same output given the same input!

2D patch embedding and 3D patch embedding have the same output? True
